In [1]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import torch
import pyro
import pyro.distributions as pdist
import pyro.contrib.gp as gp
from pyro.infer import SVI, Trace_ELBO
from pyro.optim import Adam
import math
import numpy as np


In [3]:
import torch
import torch.optim as optim

def func(X):
    return -torch.sin(6*math.pi*X)**2+6*X**2-5*X**4+3/2

# Data
X = torch.tensor(np.linspace(0, 1, 30))
y = func(X)

# Define your model
def model(X, theta):
    # This is a simple linear regression model
    return X @ theta

# Define your prior
def prior(theta):
    # Assuming a Gaussian prior
    return torch.exp(-0.5 * torch.sum(theta**2))

# Define your likelihood
def likelihood(y, y_hat):
    # Assuming normally distributed residuals
    return torch.exp(-0.5 * torch.sum((y - y_hat)**2))

# Define your posterior
def posterior(theta, X, y):
    y_hat = model(X, theta)
    return likelihood(y, y_hat) * prior(theta)

# Define a function to minimize (negative log posterior)
def neg_log_posterior(theta, X, y):
    return -torch.log(posterior(theta, X, y))

# Initialize your parameters
theta = torch.zeros(X.shape[1], requires_grad=True)

# Use a gradient-based optimizer
optimizer = optim.Adam([theta], lr=0.1)

# Training loop
for i in range(1000):
    optimizer.zero_grad()
    loss = neg_log_posterior(theta, X, y)
    loss.backward()
    optimizer.step()

print(f"MAP estimate: {theta.detach().numpy()}")


IndexError: tuple index out of range

In [ ]:
def func(X):
    return -torch.sin(6*math.pi*X)**2+6*X**2-5*X**4+3/2

# Data
X = torch.tensor(np.linspace(0, 1, 30))
y = func(X)

# Define Model
kernel = gp.kernels.Periodic(input_dim=1)  # could add a linear kernel
kernel.lengthscale = pyro.nn.PyroSample(pdist.LogNormal(0,1))
kernel.variance = pyro.nn.PyroSample(pdist.LogNormal(0,1))
gpr = gp.models.GPRegression(X, y, kernel, noise=torch.tensor(1e-4))  # the noise level


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/3, random_state=42)
gpr.set_data(X_train, y_train)